# Swept-power spectrum monitoring
### Data file summary

In [ ]:
# This notebook supports input parameters for automatic report generation. The parameters must be variables in this
# cell, which has a special 'parameters' tag.
DATA_ROOT = r'G:\Shared drives\Covid-19 Spectrum Monitoring\Data'

HISTOGRAM_RESOLUTION_SWEEPS = 100
HISTOGRAM_POWER_LOW = -99
HISTOGRAM_POWER_HIGH = -21

figure_format = 'svg'

# report.py will make separate reports for each combination of the below parameters
# available in the dataset.
site = 10#'pfQlhd1e'
year = 2020
month = 7

# One site, one month

In [ ]:
from environment import *
import figures
import histogram_analysis

bounds = (HISTOGRAM_POWER_LOW, HISTOGRAM_POWER_HIGH)
set_matplotlib_formats(figure_format)

In [ ]:
hists = histogram_analysis.read_histogram(
    Path(DATA_ROOT)/'histogram.parquet',
    site=site,
    year=year,
    month=month,
    duration=dict(months=1)
)

# # there is slight error in the achieved frequency; map intended nearby frequency
# # to achieved frequency
fc_map = dict(zip(np.round(hists.index.levels[0],1),hists.index.levels[0]))

# daily_hists = hists.groupby(['Frequency', pd.Grouper(level='Time', freq='H')]).sum()

display(histogram_analysis.hists_summary_table(hists, site))
display(widgets.HTML(f'This report was produced {time.strftime(time_format)}'));

### LTE Uplink Bands

In [ ]:
for fc in fc_lte_ul:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]],
        title=f'Sample histograms by time ({fc} MHz, site {site})',
        bounds=bounds
    )

### LTE Downlink Bands

In [ ]:
for fc in fc_lte_dl:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]],
        title=f'Sample histograms by time ({fc} MHz, site {site})',
        bounds=bounds
    )

### 2.4 GHz ISM Band

In [ ]:
for fc in fc_ism:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]],
        title=f'Sample histograms by time ({fc} MHz, site {site})',
        bounds=bounds
    )

### 5 GHz U-NII Bands

In [ ]:
for fc in fc_unii:
    ax = figures.plot_power_histogram_heatmap(
        hists.loc[fc_map[fc]],
        title=f'Sample histograms by time ({fc} MHz, site {site})',
        bounds=bounds
    )

## Validation Checks

The distribution by time helps identify transients in the distribution.  A steady noise floor stronger than -105 dBm/4 MHz may indicate that the noise calibration needs to be repeated, or the presence of steady, powerful signal overload in another band. Intermittent samples above this level suggest intermittent overload in another band.

In [ ]:
ax = figures.plot_power_histogram_heatmap(
    hists.loc[fc_quiet],
    title=f'Sample histograms by time (2695 MHz, site {site})',
    bounds=(-140,-60)
)

In [ ]:
# ### IQ sample peak
# We expect undesirable compression effects when power is sustained at -30 dBm/4 MHz or above. We are not yet sure what peak IQ sample level will correspond with compression in this dataset.

# # plot the maximum instantaneous sample power at each frequency
# fig, ax = subplots()
# sample_maxima = peak_power.reset_index().pivot(columns='Frequency', values='Sample peak').max(axis=0)
# (10*log10(sample_maxima)).plot(lw=0,marker='.',ax=ax)
# ylabel(f'Max sample power (dBm/{int(metadata["sample_rate"]/1e6)} MHz)')
# xlabel('Center frequency (MHz)');